In [3]:
# %reset
import sys, os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION']=".97"
# os.environ['XLA_PYTHON_CLIENT_PREALLOCATE']="False"

# Change the current working directory to the desired path
os.chdir('/mnt/home/spandey/ceph/GODMAX/src/')
from jax.config import config
config.update("jax_enable_x64", True)
import numpy as np
import jax.numpy as jnp
import colossus 
from jax import vmap, grad
%matplotlib inline
import matplotlib.pyplot as pl
pl.rc('text', usetex=True)
# Palatino
pl.rc('font', family='DejaVu Sans')
#import pyccl as ccl
import pickle as pk
import scipy.interpolate as interp

%load_ext autoreload
%autoreload 2

import warnings

# Suppress RuntimeWarnings
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning)




/tmp/ipykernel_584004/3064035656.py:9: DeprecationWarning: Accessing jax.config via the jax.config submodule is deprecated.
  from jax.config import config


In [4]:
cosmo_params_dict = {'flat': True, 'H0': 70.0, 'Om0': 0.315, 'Ob0': 0.0463, 'sigma8': 0.821, 'ns': 0.972, 'w0':-1.0}
sim_params_dict = {}
sim_params_dict['nfw_trunc'] = True
sim_params_dict['gamma_rhogas'] = 2.0
sim_params_dict['delta_rhogas'] = 7.0
# sim_params_dict['theta_co'] = 0.01
# sim_params_dict['theta_ej'] = 1.0

sim_params_dict['theta_co_0'] = 0.01
sim_params_dict['log10_Mstar0_theta_co'] = 15.0
sim_params_dict['nu_theta_co_M'] = 0.0
sim_params_dict['nu_theta_co_z'] = 0.0

sim_params_dict['theta_ej_0'] = 2.0
sim_params_dict['log10_Mstar0_theta_ej'] = 15.0
sim_params_dict['nu_theta_ej_M'] = -0.25
sim_params_dict['nu_theta_ej_z'] = 0.5

sim_params_dict['nu_theta_co_c'] = 0.0
sim_params_dict['nu_theta_ej_c'] = 0.0

sim_params_dict['log10_Mc0'] = 15.1
sim_params_dict['log10_Mstar0'] = 14.0
sim_params_dict['mu_beta'] = 0.1
sim_params_dict['nu_z'] = -5.0
sim_params_dict['nu_M'] = 0.0

sim_params_dict['eta_star'] = 0.3
sim_params_dict['eta_cga'] = 0.6


sim_params_dict['neg_bhse_plus_1'] = 0.833
sim_params_dict['A_starcga'] = 0.09
sim_params_dict['log10_M1_starcga'] = 11.4
sim_params_dict['epsilon_rt'] = 4.0


sim_params_dict['a_zeta'] = 0.3
sim_params_dict['n_zeta'] = 2
sim_params_dict['alpha_nt'] = 0.18
sim_params_dict['beta_nt'] = 0.5
sim_params_dict['n_nt'] = 0.3
sim_params_dict['cosmo'] = cosmo_params_dict


halo_params_dict = {}
halo_params_dict['rmin'], halo_params_dict['rmax'], halo_params_dict['nr'] = 5e-3, 24, 128
# halo_params_dict['zmin'], halo_params_dict['zmax'], halo_params_dict['nz'] = 0.001, 0.002, 2
# halo_params_dict['zmin'], halo_params_dict['zmax'], halo_params_dict['nz'] = 0.001, 1.0, 10
halo_params_dict['z_array'] = np.array([1e-3, 0.5, 1.0])
# halo_params_dict['z_array'] = np.array([1e-3])
# halo_params_dict['z_array'] = np.array([0.5])
# halo_params_dict['z_array'] = np.array([1.0])
halo_params_dict['nz'] = len(halo_params_dict['z_array'])
halo_params_dict['lg10_Mmin'], halo_params_dict['lg10_Mmax'], halo_params_dict['nM'] = 13.0, 15.5, 32
halo_params_dict['cmin'], halo_params_dict['cmax'], halo_params_dict['nc'] = 1, 10, 18
halo_params_dict['ellmin'], halo_params_dict['ellmax'], halo_params_dict['nell'] = 8, 2**15, 64
# try:
halo_params_dict['sig_logc_z_array'] = np.ones(halo_params_dict['nz']) * 0.01
halo_params_dict['mdef'] = '200c'
halo_params_dict['hmf_model'] = 'T10'
halo_params_dict['conc_model'] = 'Diemer15'
halo_params_dict['do_corr_2h_mm'] = True

# halo_params_dict['do_corr_2h_mm'] = False

from astropy.io import fits
df = fits.open('/mnt/home/spandey/ceph/GODMAX/data/sim_3x2pt_simulated_DV_PKproject_values_bestfit_maglim_3x2LCDM_final.fits') 
z_array = df['nz_source'].data['Z_MID']
nz_info_dict = {}
nz_info_dict['z_array'] = z_array
nz_info_dict['nbins'] = 4
nz_info_dict['nz0'] = np.maximum(df['nz_source'].data['BIN1'], 1e-4)
nz_info_dict['nz1'] = np.maximum(df['nz_source'].data['BIN2'], 1e-4)
nz_info_dict['nz2'] = np.maximum(df['nz_source'].data['BIN3'], 1e-4)
nz_info_dict['nz3'] = np.maximum(df['nz_source'].data['BIN4'], 1e-4)
analysis_dict = {}
analysis_dict['nz_info_dict'] = nz_info_dict
analysis_dict['do_sheary'] = True
analysis_dict['do_shear2pt'] = True
analysis_dict['do_yy'] = False

df_data = fits.open('/mnt/home/spandey/ceph/GODMAX/data/DES_ACT_full_data_theorycov_2.5.fits')
theta_data = df_data['compton_shear'].data['ANG'][0:20]

analysis_dict['ellmin_transf'], analysis_dict['ellmax_transf'], analysis_dict['nell_transf'] = 8, 2**15, 16384
analysis_dict['angles_data_array'] = jnp.array(theta_data)
analysis_dict['beam_fwhm_arcmin'] = 1.6
analysis_dict['want_like_diff'] = False
analysis_dict['calc_nfw_only'] = True





In [5]:
from colossus.cosmology import cosmology
from colossus.halo import concentration
from colossus.lss import mass_function
import scipy.interpolate as interpolate

cosmo = cosmology.setCosmology('myCosmo', **cosmo_params_dict)

# from get_corr_func_jit import get_corrfunc_BCMP
from get_BCMP_profile_jit import BCM_18_wP
import numpyro
import copy
# from jax_cosmo.scipy.interpolate import interp
import scipy.interpolate as interp
from jax import grad
from jax import vmap
from jax import value_and_grad

from jaxopt import ScipyBoundedMinimize
from jaxopt import LBFGSB



# params_vary_names = ['n_nt', 'alpha_nt', 'theta_ej_0', 'log10_Mstar0_theta_ej', 'nu_theta_ej_M', 'theta_co_0', 'nu_theta_co_M', 'nu_theta_ej_z', 'nu_z', 'log10_Mc0', 'gamma_rhogas', 'delta_rhogas', 'mu_beta', 'eta_star', 'eta_cga']
# lower_bounds = jnp.array([0.0001, -3.0, 0.00001, -8.0, -8.0, -8.0, 10.0, 0.01, 0.01, -1.0, 0.05, 0.3])
# upper_bounds = jnp.array([8.0, 3.0, 0.8, 3.0, 3.0, 3.0, 17.0, 8.0, 15.0, 0.8, 8.0, 5.0])
lower_bounds_dict = {'n_nt':0.0001, 'alpha_nt':0.01 , 'log10_Mstar0_theta_ej':11.0, 'theta_ej_0': 0.5, 'nu_theta_ej_M': -5.0, 'theta_co_0': 1e-12, 'nu_theta_co_M': -8.0, 'nu_theta_ej_z': -8.0, 'nu_z': -8.0, 'log10_Mc0': 10.0, 'gamma_rhogas': 0.2, 'delta_rhogas': 0.01, 'mu_beta': 0.0, 'eta_star': 0.001, 'eta_cga': 0.01}
upper_bounds_dict = {'n_nt':0.7,'alpha_nt':0.5 , 'log10_Mstar0_theta_ej':15.0,'theta_ej_0': 8.0, 'nu_theta_ej_M': 0.0, 'theta_co_0': 0.8, 'nu_theta_co_M': 3.0, 'nu_theta_ej_z': 5.0, 'nu_z': 8.0, 'log10_Mc0': 16.5, 'gamma_rhogas': 6.0, 'delta_rhogas': 15.0, 'mu_beta': 5.0, 'eta_star': 0.6, 'eta_cga': 1.0}


# params_vary_names = ['n_nt', 'theta_ej_0', 'nu_theta_ej_M', 'nu_theta_co_M', 'nu_theta_ej_z', 'nu_z', 'theta_co_0', 'gamma_rhogas']
# params_vary_names = ['eta_star','alpha_nt', 'theta_ej_0', 'theta_co_0', 'nu_theta_ej_M', 'nu_theta_ej_z', 'nu_z', 'log10_Mc0', 'mu_beta', 'delta_rhogas', 'gamma_rhogas']

# params_vary_names = ['theta_ej_0', 'theta_co_0', 'gamma_rhogas','mu_beta','eta_star','alpha_nt', 'log10_Mc0','nu_theta_ej_M']
params_vary_names = ['theta_ej_0', 'theta_co_0','nu_z', 'nu_theta_ej_z', 'gamma_rhogas','mu_beta', 'log10_Mc0','eta_star','alpha_nt', 'nu_theta_ej_M']
# params_vary_names = ['theta_ej_0', 'theta_co_0','nu_z', 'gamma_rhogas','mu_beta', 'log10_Mc0','eta_star','alpha_nt']
# params_vary_names = ['theta_ej_0', 'theta_co_0','nu_z', 'gamma_rhogas','nu_theta_ej_M','eta_star','alpha_nt']





/mnt/home/spandey/venv_gm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from colossus.cosmology import cosmology
# set cosmology:
cosmology.setCosmology('planck15')
from colossus.halo import concentration

cM13 = concentration.concentration(1E13, '200c', 0.0, model = 'diemer15')
cM14 = concentration.concentration(1E14, '200c', 0.0, model = 'diemer15')
cM15 = concentration.concentration(1E15, '200c', 0.0, model = 'diemer15')



In [7]:
import copy
import copy
from get_BCMP_profile_jit import BCM_18_wP
from setup_power_spectra_jit import setup_power_BCMP
from tqdm import tqdm
import scipy.integrate as integrate
h = sim_params_dict['cosmo']['H0'] / 100.0
Ob = sim_params_dict['cosmo']['Ob0']
Om = sim_params_dict['cosmo']['Om0']
import astropy.units as u
from astropy import constants as const
import jax_cosmo.background as bkgrd
import scipy.integrate as integrate
from setup_power_spectra_jit import setup_power_BCMP
sigmat = const.sigma_T
m_e = const.m_e
c = const.c
coeff = sigmat / (m_e * (c ** 2))
oneMpc_h_to_cm = (((10 ** 6)/h) * (u.pc).to(u.cm))
const_coeff = ((coeff).to(((u.kpc ** 2) / u.keV))).value







In [8]:
Y_all_data = jnp.array([0.008, 1.1, 45])



In [9]:
def get_value(x):        
    sim_params_dict_vary = copy.deepcopy(sim_params_dict)
    for jp in range(len(params_vary_names)):
        sim_params_dict_vary[params_vary_names[jp]] = x[jp]

    
    # Now that params are defined, here is the forward model
    # get_corrfunc_BCMP_obj = get_corrfunc_BCMP(sim_params_dict_vary, halo_params_dict, analysis_dict,num_points_trapz_int=32)
    BCMP_test_vary = BCM_18_wP(sim_params_dict_vary, halo_params_dict)
    # theory = BCMP_test_vary.rho_dmb_mat

    # rho_dmb = BCMP_test_vary.rho_dmb_mat_physical
    # rho_gas = BCMP_test_vary.rho_gas_mat_physical
    # ne_gas = BCMP_test_vary.ne_mat_physical    
    # Pe_gas = BCMP_test_vary.Pe_mat_physical
    
    z_test = 0.0   
    z_array = BCMP_test_vary.z_array   
    scale_fac_a_array = 1./(1. + z_array)
    indz = jnp.argmin(jnp.abs(z_array - z_test))

    M_array = BCMP_test_vary.M_array
    c_array = BCMP_test_vary.conc_array 

    indM1 = jnp.argmin(jnp.abs(M_array - 1e13))
    indM2 = jnp.argmin(jnp.abs(M_array - 1e14))
    indM3 = jnp.argmin(jnp.abs(M_array - 1e15))

    indc1 = jnp.argmin(jnp.abs(c_array - cM13))
    indc2 = jnp.argmin(jnp.abs(c_array - cM14))
    indc3 = jnp.argmin(jnp.abs(c_array - cM15))

    # Y_all_theory = 
    for ji in range(3):
        if ji == 0:
            indM = indM1
            indc = indc1
        elif ji == 1:
            indM = indM2
            indc = indc2
        elif ji == 2:
            indM = indM3
            indc = indc3

        r200c_Mz = BCMP_test_vary.r200c_mat[indM, indz]

        log_r_array = jnp.linspace(jnp.log(0.05*r200c_Mz), jnp.log(r200c_Mz), 100)

        # indsel_r = np.where(BCMP_test.r_array < r200c_Mz)[0]
        # r_array_j = BCMP_test.r_array[indsel_r]
        r_array_j = jnp.exp(log_r_array)

        # rho_cga_j = BCMP_test_vary.rho_cga_mat[:,indc, indz, indM]
        # rho_clm_j = BCMP_test_vary.rho_clm_mat[:,indc, indz, indM]
        # rho_gas_j = BCMP_test.rho_gas_mat[:,indc, indz, indM]
        # rho_dmb_j = BCMP_test.rho_dmb_mat[:,indc, indz, indM]
        # rho_nfw_j = BCMP_test.rho_nfw_mat[:,indc, indz, indM]
        Pe_mat_j = BCMP_test_vary.Pe_mat_physical[:,indc, indz, indM]

        # interpolate all into log_r_array:
        # rho_cga_j = np.interp(log_r_array, np.log(BCMP_test.r_array), rho_cga_j, left=0.0, right=0.0)
        # rho_clm_j = np.interp(log_r_array, np.log(BCMP_test.r_array), rho_clm_j, left=0.0, right=0.0)
        # rho_gas_j = np.interp(log_r_array, np.log(BCMP_test.r_array), rho_gas_j, left=0.0, right=0.0)
        # rho_dmb_j = np.interp(log_r_array, np.log(BCMP_test.r_array), rho_dmb_j, left=0.0, right=0.0)
        # rho_nfw_j = np.interp(log_r_array, np.log(BCMP_test.r_array), rho_nfw_j, left=0.0, right=0.0)
        Pe_mat_j = jnp.exp(jnp.interp(log_r_array, jnp.log(BCMP_test_vary.r_array), jnp.log(Pe_mat_j), left=0.0, right=0.0))

        # Mcga_int_j = integrate.simps(4 * np.pi * rho_cga_j * r_array_j**2 * r_array_j, np.log(r_array_j))
        # Mclm_int_j = integrate.simps(4 * np.pi * rho_clm_j * r_array_j**2 * r_array_j, np.log(r_array_j))
        # Mgas_int_j = integrate.simps(4 * np.pi * rho_gas_j * r_array_j**2 * r_array_j, np.log(r_array_j))
        # Mdmb_int_j = integrate.simps(4 * np.pi * rho_dmb_j * r_array_j**2 * r_array_j, np.log(r_array_j))
        # Mnfw_int_j = integrate.simps(4 * np.pi * rho_nfw_j * r_array_j**2 * r_array_j, np.log(r_array_j))
        Pe_int_j = jnp.trapz(4 * np.pi * Pe_mat_j * r_array_j**2 * r_array_j, jnp.log(r_array_j)) * (oneMpc_h_to_cm**3)
        Y_int_j = const_coeff * Pe_int_j
        if ji == 0:
            Y1 = Y_int_j
        elif ji == 1:
            Y2 = Y_int_j
        elif ji == 2:
            Y3 = Y_int_j
        
    Y_all_theory = jnp.array([Y1, Y2, Y3])
    print(Y_all_theory)
    # theory_combined = jnp.log(theory_combined)
    chi2 = jnp.sqrt(jnp.sum((Y_all_data - Y_all_theory)**2))
    # print(chi2)

    return chi2




In [10]:
lbfgsb = ScipyBoundedMinimize(fun=value_and_grad(get_value), value_and_grad=True, method="l-bfgs-b", tol=1e-9, maxiter=100)

lower_bounds = []
upper_bounds = []
for jp in range(len(params_vary_names)):
    lower_bounds.append(lower_bounds_dict[params_vary_names[jp]])
    upper_bounds.append(upper_bounds_dict[params_vary_names[jp]])

lower_bounds = jnp.array(lower_bounds)
upper_bounds = jnp.array(upper_bounds)

bounds = (lower_bounds, upper_bounds)

w_init = []
for jp in range(len(params_vary_names)):
    w_init.append(sim_params_dict[params_vary_names[jp]])
w_init = jnp.array(w_init)


res = lbfgsb.run(w_init, bounds=bounds)
# lbfgs_sol = lbfgsb.params

# res.
params, state = res


Traced<ShapedArray(float64[3])>with<JVPTrace(level=3/1)> with
  primal = Traced<ShapedArray(float64[3])>with<DynamicJaxprTrace(level=1/1)>
  tangent = Traced<ShapedArray(float64[3])>with<JaxprTrace(level=2/1)> with
    pval = (ShapedArray(float64[3]), None)
    recipe = JaxprEqnRecipe(eqn_id=<object object at 0x1554c145e690>, in_tracers=(Traced<ShapedArray(float64[1]):JaxprTrace(level=2/1)>, Traced<ShapedArray(float64[1]):JaxprTrace(level=2/1)>, Traced<ShapedArray(float64[1]):JaxprTrace(level=2/1)>), out_tracer_refs=[<weakref at 0x1554c1206930; to 'JaxprTracer' at 0x1554c12068f0>], out_avals=[ShapedArray(float64[3])], primitive=concatenate, params={'dimension': 0}, effects=frozenset(), source_info=SourceInfo(traceback=<jaxlib.xla_extension.Traceback object at 0x1554c120c6f0>, name_stack=NameStack(stack=(Transform(name='jvp'),))))


2024-01-29 17:47:16.372034: W external/tsl/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 33.02GiB (rounded to 35457384192)requested by op 
2024-01-29 17:47:16.372116: W external/tsl/tsl/framework/bfc_allocator.cc:497] *___________________________________________________________________________________________________
2024-01-29 17:47:16.372857: E external/xla/xla/pjrt/pjrt_stream_executor_client.cc:2732] Execution of replica 0 failed: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 35457384176 bytes.
BufferAssignment OOM Debugging.
BufferAssignment stats:
             parameter allocation:        80B
              constant allocation:       236B
        maybe_live_out allocation:       104B
     preallocated temp allocation:   33.02GiB
                 total allocation:   33.02GiB
Peak buffers:
	Buffer 1:
		Size: 6.75GiB
		Operator: op_name="jit(value_and_grad_without_aux)/jit(main)/transpose(jvp(vmap(vmap(vmap(vmap(jit(get_Md

XlaRuntimeError: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 35457384176 bytes.
BufferAssignment OOM Debugging.
BufferAssignment stats:
             parameter allocation:        80B
              constant allocation:       236B
        maybe_live_out allocation:       104B
     preallocated temp allocation:   33.02GiB
                 total allocation:   33.02GiB
Peak buffers:
	Buffer 1:
		Size: 6.75GiB
		Operator: op_name="jit(value_and_grad_without_aux)/jit(main)/transpose(jvp(vmap(vmap(vmap(vmap(jit(get_Mdmb)))))))/vmap(jit(get_rho_dmb))/jit(get_rho_clm)/jit(get_rho_nfw_normed)/broadcast_in_dim[shape=(32, 3, 18, 128, 64, 64) broadcast_dimensions=()]" source_file="/mnt/ceph/users/spandey/GODMAX/src/get_BCMP_profile_jit.py" source_line=344
		XLA Label: fusion
		Shape: f64[64,64,32,3,18,128]
		==========================

	Buffer 2:
		Size: 3.38GiB
		Operator: op_name="jit(value_and_grad_without_aux)/jit(main)/jvp(vmap(vmap(vmap(vmap(jit(get_zeta))))))/vmap(jit(get_Mgas))

In [ ]:
params



Array([ 2.00e+00,  1.00e-02, -5.00e+00,  5.00e-01,  2.00e+00,  1.00e-01,
        1.51e+01,  3.00e-01,  1.80e-01, -2.50e-01], dtype=float64)

In [ ]:
w_init = jnp.array(w_init)
w_init


Array([ 2.00e+00,  1.00e-02, -5.00e+00,  5.00e-01,  2.00e+00,  1.00e-01,
        1.51e+01,  3.00e-01,  1.80e-01, -2.50e-01], dtype=float64)

In [ ]:
w_init = []
for jp in range(len(params_vary_names)):
    w_init.append(sim_params_dict[params_vary_names[jp]])
w_init = jnp.array(w_init)




In [ ]:
get_value(w_init)




[1.00079542e-02 1.22667821e+00 1.62956276e+02]


Array(117.95634409, dtype=float64)

In [ ]:
# !pip install --upgrade pip
Y_all_data


Array([8.0e-03, 1.1e+00, 4.5e+01], dtype=float64)